In [1]:
import pandas as pd
import re


file_path = 'final_sum_test.csv'
data = pd.read_csv(file_path)


def clean_text(text):
    cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text


data['Original'] = data['Original'].apply(clean_text)
data['Summary.1'] = data['Summary.1'].apply(clean_text)

cleaned_file_path = 'cleaned_final_sum_test_separate_columns.csv'
data[['Original', 'Summary.1']].to_csv(cleaned_file_path, index=False)



KeyboardInterrupt



In [10]:
!pip install datasets rouge_score evaluate


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 995.2 kB/s eta 0:00:00a 0:00:01


In [14]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from arabert.preprocess import ArabertPreprocessor
import torch
from datasets import Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer
from arabert.preprocess import ArabertPreprocessor
import pandas as pd

In [15]:
import pandas as pd

# Load dataset from CSV file
data_path = "final_merged_summary.csv"
df = pd.read_csv(data_path)

# Display the first few rows of the dataset and column names
print(df.head())
print(df.columns)


                                            original  \
0  وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...   
1  ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...   
2  قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...   
3  دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...   
4  السوريون يصرون على استقلال بلادهم : و مثلما رف...   

                                             summary  
0  يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...  
1  دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...  
2  أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...  
3  مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...  
4  الشعب السوري يصر على استقلال بلدهم من السيطرة ...  
Index(['original', 'summary'], dtype='object')


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
from arabert.preprocess import ArabertPreprocessor
import torch
import evaluate

# Load model and tokenizer
model_name = "malmarjeh/t5-arabic-text-summarization"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load dataset from CSV file
data_path = "final_merged_summary.csv"
df = pd.read_csv(data_path)

# Display the first few rows of the dataset and column names
print(df.head())
print(df.columns)

# Rename columns to match the expected column names in preprocess_function
df.columns = ['original', 'summary']

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Preprocess data
arabert_prep = ArabertPreprocessor(model_name.split('/')[1])

def preprocess_function(examples):
    # Preprocess the input text
    inputs = ["summarize: " + arabert_prep.preprocess(doc) for doc in examples["original"]]
    
    # Ensure summaries are in the correct format and convert to strings
    summaries = [str(summary) for summary in examples["summary"]]
    
    # Tokenize inputs and summaries
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(summaries, max_length=150, truncation=True, padding="max_length")
    
    # Add labels to model_inputs
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Load ROUGE metric
rouge = evaluate.load("rouge")

def compute_rouge(model, dataset):
    model.eval()
    predictions, references = [], []
    
    for example in dataset:
        input_ids = torch.tensor(example['input_ids']).unsqueeze(0)
        outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reference = tokenizer.decode(example['labels'], skip_special_tokens=True)
        
        predictions.append(prediction)
        references.append(reference)
    
    rouge_output = rouge.compute(predictions=predictions, references=references)
    return rouge_output

# Evaluate the original model
original_model_scores = compute_rouge(model, tokenized_dataset)
print("Original Model ROUGE Scores:", original_model_scores)


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


                                            original  \
0  وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...   
1  ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...   
2  قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...   
3  دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...   
4  السوريون يصرون على استقلال بلادهم : و مثلما رف...   

                                             summary  
0  يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...  
1  دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...  
2  أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...  
3  مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...  
4  الشعب السوري يصر على استقلال بلدهم من السيطرة ...  
Index(['original', 'summary'], dtype='object')


Map:   0%|          | 0/275274 [00:00<?, ? examples/s]

In [ ]:
# Load the last checkpoint
checkpoint_directory = r"C:\Users\USER\Desktop\Bahaa_GP\t5_weights\3"
last_checkpoint_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_directory)
last_checkpoint_tokenizer = AutoTokenizer.from_pretrained(checkpoint_directory)

# Evaluate the last checkpoint model
last_checkpoint_scores = compute_rouge(last_checkpoint_model, tokenized_dataset)
print("Last Checkpoint ROUGE Scores:", last_checkpoint_scores)